In [1]:
from astropy.utils.data import download_file
from lightkurve import search_lightcurve
from lcviz import LCviz

lc = search_lightcurve("HAT-P-11", mission="Kepler", cadence="long", quarter=10).download()

lcviz = LCviz()
lcviz.load_data(lc)
lcviz.show()

/Users/kconroy/miniconda3/envs/lcviz2/lib/python3.11/site-packages/glue/viewers/common/qt/__init__.py:3: GlueDeprecationWarning: Importing from glue.viewers.common.qt is deprecated, use glue_qt.viewers.common) instead
  warnings.warn('Importing from glue.viewers.common.qt is deprecated, use glue_qt.viewers.common) instead', GlueDeprecationWarning)
/Users/kconroy/miniconda3/envs/lcviz2/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Application(config='lcviz', docs_link='https://lcviz.readthedocs.io', events=['call_viewer_method', 'close_sna…

Each plugin in the sidebar also has its own user-friendly API to allow creating reproducible workflows in Jupyter.  These are accessible via the `lcviz.plugins` dictionary, where the keys are the names of the plugins:

In [2]:
print(lcviz.plugins)

{'Metadata': <Metadata API>, 'Plot Options': <Plot Options API>, 'Subset Tools': <Subset Tools API>, 'Markers': <Markers API>, 'Flatten': <Flatten API>, 'Frequency Analysis': <Frequency Analysis API>, 'Ephemeris': <Ephemeris API>, 'Export Plot': <Export Plot API>}


We can now access the API of any of these plugins.  For this example, we'll use the [Flatten plugin](https://lcviz.readthedocs.io/en/latest/plugins.html#flatten) to flatten the light curve from the [Flatten plugin API](https://lcviz.readthedocs.io/en/latest/api/lcviz.plugins.flatten.flatten.Flatten.html).

Links to the docs are also available from the plugin UI, which can be shown either in the tray of the app or in its own cell, via the "Learn More" link.

In [3]:
flatten = lcviz.plugins['Flatten']
print(flatten)

<Flatten API>


the basic API docs can also be shown in the notebook (with `help(plugin)` or `plugin?`) and the list of available attributes and methods is accessible with `dir(plugin)`:

In [4]:
help(flatten)

Help on PluginUserApi in module jdaviz.core.user_api:

<Flatten API>
    This is an API wrapper around an internal plugin.  For a full list of attributes/methods,
    call dir(plugin_object) and for help on any of those methods,
    call help(plugin_object.attribute).
    
    For example::
      help(plugin_object.show)
    
    
    
    
    See the :ref:`Flatten Plugin Documentation <flatten>` for more details.
    
    Only the following attributes and methods are available through the
    public plugin API.
    
    * ``show_live_preview``
    * ``default_to_overwrite``
    * ``dataset`` (:class:`~jdaviz.core.template_mixin.DatasetSelect`):
      Dataset to flatten.
    * ``add_results`` (:class:`~jdaviz.core.template_mixin.AddResults`)
    * ``window_length``
    * ``polyorder``
    * ``break_tolerance``
    * ``niters``
    * ``sigma``
    * ``unnormalize``
    * :meth:`flatten`



In [5]:
dir(flatten)

['add_results',
 'as_active',
 'break_tolerance',
 'dataset',
 'default_to_overwrite',
 'flatten',
 'keep_active',
 'niters',
 'open_in_tray',
 'polyorder',
 'show',
 'show_live_preview',
 'sigma',
 'unnormalize',
 'window_length']

We can see here that all the widgets in the plugin UI have corresponding attributes or methods in the plugin API.

In [6]:
print(flatten.dataset)

<selected='KIC 10748390[pdcsap_flux]' choices=['KIC 10748390[pdcsap_flux]']>


In [7]:
flatten.polyorder = 4
flatten.niters = 5
flatten.sigma = 3
flatten.unnormalize = True

As shown in the API docs, calling `.flatten()` returns both the flattened and trend light curve as `lightkurve` objects, and would load that entry back into the app by default.

In [8]:
help(flatten.flatten)

Help on method flatten in module lcviz.plugins.flatten.flatten:

flatten(add_data=True) method of lcviz.plugins.flatten.flatten.Flatten instance
    Flatten the input light curve (``dataset``) using lightkurve.flatten.
    
    Parameters
    ----------
    add_data : bool
        Whether to add the resulting trace to the application, according to the options
        defined in the plugin.
    
    Returns
    -------
    output_lc : `~lightkurve.LightCurve`
        The flattened light curve.
    trend_lc : `~lightkurve.LightCurve`
        The trend used to flatten the light curve.



In [9]:
flattened_lc, trend_lc = flatten.flatten(add_data=False)
print(type(flattened_lc))

<class 'lightkurve.lightcurve.LightCurve'>


To load the results back as a data entry, we could pass `add_data=True` instead, in which case a new entry will be added to the app according to the label and viewer defined in `add_results`.

In [10]:
print(flatten.add_results)

<AddResults label='KIC 10748390[pdcsap_flux]', auto=True, viewer=flux-vs-time>


In the case of the flattening plugin, we can see that this will overwrite the input data entry.  To avoid that, we can set the label to something else or toggle the setting to generate an automatic label that will not overwrite.

In [11]:
flatten.default_to_overwrite = False

In [12]:
flatten.add_results

<AddResults label='KIC 10748390[pdcsap_flux] (flattened)', auto=True, viewer=flux-vs-time>

In [13]:
flattened_lc, trend_lc = flatten.flatten(add_data=True)

Now that a new light curve has been added to the app, dataset dropdowns (and their respective UI elements) are updated to show the new entry (where applicable):

In [14]:
print(flatten.dataset)

<selected='KIC 10748390[pdcsap_flux]' choices=['KIC 10748390[pdcsap_flux]', 'KIC 10748390[pdcsap_flux] (flattened)']>
